# Setup

In [ ]:
!pip install optuna

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import glob
import numpy as np
import os
import pandas as pd
import random

import matplotlib.pyplot as plt

import optuna

In [ ]:

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Dataset

In [ ]:


class SpeedDataset(Dataset):
    def __init__(self, directory_paths, sequence_length):
        self.sequence_length = sequence_length
        sequences, labels, timestamps = self.load_data_from_directories(directory_paths)
        self.data = torch.tensor(sequences, dtype=torch.float32).transpose(1, 2)
        self.labels = torch.tensor(labels, dtype=torch.float32)
        self.timestamps = timestamps

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

    def load_data_from_directories(self, directory_paths):
        all_sequences = None
        all_labels = None
        all_timestamps = None
        for dir_path in directory_paths:
            for filename in os.listdir(dir_path):
                if filename.endswith(".csv"):
                    file_path = os.path.join(dir_path, filename)
                    data = pd.read_csv(file_path, header=None, names=['timestamp', 'x_acc', 'y_acc', 'z_acc', 'gps_speed'])
                    sequences, labels, timestamps = self.create_sequences(data, self.sequence_length)
                    if all_sequences is None:
                        all_sequences = sequences
                        all_labels = labels
                        all_timestamps = timestamps
                    else:
                        all_sequences = np.concatenate((all_sequences, sequences), axis=0)
                        all_labels = np.concatenate((all_labels, labels), axis=0)
                        all_timestamps = np.concatenate((all_timestamps, timestamps), axis=0)
        return all_sequences, all_labels, all_timestamps

    def create_sequences(self, data, sequence_length):
        sequences = []
        labels = []
        timestamps = []

        for i in range(len(data) - sequence_length):
            seq = data.iloc[i:i+sequence_length][['x_acc', 'y_acc', 'z_acc']].values
            label = data.iloc[i:i+sequence_length]['gps_speed'].mean()
            sequences.append(seq)
            labels.append(label)
            timestamps.append(data.iloc[i]['timestamp'])

        return np.array(sequences), np.array(labels), np.array(timestamps)




In [ ]:
# Create training and testing datasets
train_dir = ['/content/drive/Shareddrives/CS229/Data/FinalSplits/train_axel/']
eval_dir = ['/content/drive/Shareddrives/CS229/Data/FinalSplits/eval_axel/']
test_dir = ['/content/drive/Shareddrives/CS229/Data/FinalSplits/test_axel/']

# Model

In [ ]:


class CNNModel(nn.Module):
    def __init__(self, conv1_filters, conv2_filters, dropout_rate, kernel_size):
        super(CNNModel, self).__init__()
        sequence_length = 40
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=conv1_filters, kernel_size=kernel_size, stride=1, padding=kernel_size // 2)
        self.conv2 = nn.Conv1d(in_channels=conv1_filters, out_channels=conv2_filters, kernel_size=kernel_size, stride=1, padding=kernel_size // 2)
        self.fc1 = nn.Linear(conv2_filters * sequence_length, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x




# Optimize Hyperparameters


In [ ]:
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    sequence_length = 40
    conv1_filters = trial.suggest_int("conv1_filters", 8, 64, step=8)
    conv2_filters = trial.suggest_int("conv2_filters", 16, 128, step=16)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    kernel_size = trial.suggest_int("kernel_size", 3, 7, step=2)


    model = CNNModel(conv1_filters, conv2_filters, dropout_rate, kernel_size).to(device)

    train_dataset = SpeedDataset(train_dir, sequence_length)
    eval_dataset = SpeedDataset(eval_dir, sequence_length)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    X_eval_tensor = eval_dataset.data.to(device)
    y_eval_tensor = eval_dataset.labels.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    max_epochs = 1000
    patience = 50
    best_eval_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(max_epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_tensor = X_batch.to(device)
            y_tensor = y_batch.to(device)

            optimizer.zero_grad()
            predictions = model(X_tensor).squeeze()
            loss = criterion(predictions, y_tensor)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            eval_predictions = model(X_eval_tensor).squeeze()
            eval_loss = criterion(eval_predictions, y_eval_tensor)

        if eval_loss.item() < best_eval_loss:
            best_eval_loss = eval_loss.item()
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            break

    return best_eval_loss



In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)
print("Best Hyperparameters:", study.best_params)



# Final training (using both training and eval data)


In [ ]:
lr = study.best_params["learning_rate"]
sequence_length = 40
conv1_filters = study.best_params["conv1_filters"]
conv2_filters = study.best_params["conv2_filters"]
dropout_rate = study.best_params["dropout_rate"]
kernel_size = study.best_params["kernel_size"]

batch_size = 32

model = CNNModel(conv1_filters, conv2_filters, dropout_rate, kernel_size).to(device)

train_dataset = SpeedDataset(train_dir + eval_dir, sequence_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

max_epochs = 10000
epsillon = 0.001

total_loss = 0.0
model.train()
for epoch in range(max_epochs):

    previous_loss = total_loss
    total_loss = 0.0
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        X_tensor = X_batch.to(device)
        y_tensor = y_batch.to(device)

        predictions = model(X_tensor)
        predictions = torch.squeeze(predictions)
        loss = criterion(predictions, y_tensor)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if abs(previous_loss - total_loss) < epsillon:
      break

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{max_epochs}, Total Loss: {total_loss:.4f}")


In [ ]:


learning_rate = 0.0001
sequence_length = 40
conv1_filters = 32
conv2_filters = 16
dropout_rate = 0.0
kernel_size = 9

train_dir = [
    '/content/drive/Shareddrives/CS229/Data/FinalSplits/train_axel/',
    '/content/drive/Shareddrives/CS229/Data/FinalSplits/train_jengchi/',
    ]
eval_dir = ['/content/drive/Shareddrives/CS229/Data/FinalSplits/eval_axel/']

model = CNNModel(conv1_filters, conv2_filters, dropout_rate, kernel_size).to(device)

train_dataset = SpeedDataset(train_dir, sequence_length)
eval_dataset = SpeedDataset(eval_dir, sequence_length)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
X_eval_tensor = eval_dataset.data.to(device)
y_eval_tensor = eval_dataset.labels.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

max_epochs = 1000
patience = 8
best_eval_loss = float('inf')
epochs_without_improvement = 0

train_losses = []
eval_losses = []

for epoch in range(max_epochs):

    model.train()
    total_loss = 0.0
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        X_tensor = X_batch.to(device)
        y_tensor = y_batch.to(device)

        optimizer.zero_grad()
        predictions = model(X_tensor).squeeze()
        loss = criterion(predictions, y_tensor)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    model.eval()
    with torch.no_grad():
        eval_predictions = model(X_eval_tensor).squeeze()
        eval_loss = criterion(eval_predictions, y_eval_tensor)
        eval_losses.append(eval_loss.item())

    print(f"Epoch {epoch+1}/{max_epochs}, Train Loss: {total_loss:.4f}, Eval Loss: {eval_loss.item():.4f}")

    if eval_loss.item() < best_eval_loss:
        best_eval_loss = eval_loss.item()
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

plt.figure(figsize=(12, 6))
plt.title('Training and Validation Losses')
plt.plot(train_losses, label='Training Loss', color='blue')
plt.plot(eval_losses, label='Validation Loss', color='red')
plt.xlabel('Epoch')
plt.ylabel('Loss')


# Evaluation

### Calculate Metrics


In [ ]:
test_dir = ['/content/drive/Shareddrives/CS229/Data/FinalSplits/test_axel/']

train_dataset = SpeedDataset(train_dir, sequence_length)
test_dataset = SpeedDataset(test_dir, sequence_length)

model.eval()

predicted_train = model(train_dataset.data.to(device)).detach()
predicted_train = torch.squeeze(predicted_train).to(device)

predicted_test = model(test_dataset.data.to(device)).detach()
predicted_test = torch.squeeze(predicted_test).to(device)

mse_train = (np.square(predicted_train.to("cpu") - train_dataset.labels.numpy())).mean(axis=0)
mse_test = (np.square(predicted_test.to("cpu") - test_dataset.labels.numpy())).mean(axis=0)

print(f"MSE Train: {mse_train}")
print(f"MSE Test: {mse_test}")

### Plot

In [ ]:
predicted_test = model(test_dataset.data.to(device)).detach().to("cpu").numpy()

plt.figure(figsize=(12, 6))
plt.plot(test_dataset.timestamps, test_dataset.labels, label='Ground Truth GPS Speed', color='blue')
plt.plot(test_dataset.timestamps, predicted_test, label='Predicted GPS Speed', color='red')
plt.xlabel('Timestamp (ms since start)')
plt.ylabel('Speed (m/s)')
plt.title('Ground Truth vs. Predicted GPS Speed')
plt.legend()
plt.grid(True)
plt.show()